## Library imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import pandas as pd
from chromatography import *
from torch import optim, tensor
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mayavi import mlab
%matplotlib qt

## Dataset Import

In [22]:
alist = pd.read_csv('../data/GilarSample.csv')
# GilarSample
# Peterpeptides
# Roca
# Peter32

## Useful functions definition

In [3]:
def step_decay(lr, iteration, num_episodes, steps=10, decay_factor=0.8):
    if iteration % (num_episodes // steps) == 0:
        return lr * decay_factor
    
    return lr

In [4]:
def loss_field(exp, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], [0.25, 3000000])
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses

In [ ]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
pol = Policy(2, sigma_max = 0.2)
taus = [.25, 3000000]
losses, best_program, mus, sigmas, n_pars, samples, mu_grads, sigma_grads = reinforce(
        exp, 
        pol, 
        taus, 
        num_episodes=3000, 
        batch_size=10,
        optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65),
        lr=.1, 
        print_every=100,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=0.5),
        baseline=0.55,
        max_norm = 1.,
    )

In [ ]:
exp.reset()
exp.run_all(pol.mu.detach().numpy(), taus)
exp.print_analytes(rc=(7, 5))
exp.loss()

In [ ]:
plt.plot(mus[:, 0], label='Mu: phi1')
plt.plot(mus[:, 1], label='Mu: phi2')
#plt.plot(mus[:, 2], label='Mu: phi3')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas[:, 0], label='Sigma: phi1')
plt.plot(sigmas[:, 1], label='Sigma: phi2')
#plt.plot(sigmas[:, 2], label='Sigma: phi3')
plt.legend()
plt.show()

In [ ]:
plt.plot(losses)

In [ ]:
exp.reset()
exp.run_all(best_program.numpy(), taus)
exp.print_analytes(rc=(7, 5), title="Best Result")
exp.loss()

In [ ]:
X, Y, Loss_field = loss_field(exp, N = 100)

In [ ]:
fig, ax = plt.subplots()
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

line, = plt.plot(mus[:, 0], mus[:, 1], color='k')
scat = plt.scatter(samples[:, 0], samples[:, 1], c='r', s=3)

def update(num, mu, samples, line, scat):
    line.set_data(mu[num-2:num, 0], mu[num-2:num, 1])
    line.axes.axis([0, 1, 0, 1])
    scat.set_offsets(samples[(num-2)*10: num*10 - 10, :])
    return line,

ani = animation.FuncAnimation(fig, update, len(mus[:, 0]), fargs=[mus, samples, line, scat],
                              interval=10, blit=False)
plt.show()

In [ ]:
lossess = []
for phi in np.linspace(0,.4, 500):
    exp.reset()
    exp.run_all([phi, 0.368], [.25, 3000])
    lossess.append(exp.loss())
plt.plot(np.linspace(0,.4, 500), lossess)

In [ ]:
import matplotlib.patches as patches
fig, ax = plt.subplots()
ax.set_ylim(0, 1)
ax.set_xlim(0, 1)
plt.xlabel("phi1")

plt.ylabel("phi2")
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

n = 1998
patch = patches.Arrow(mus[n, 0], mus[n, 1], mu_grads[n, 0], mu_grads[n, 1], width=0.02, color='k')
ax.add_patch(patch)
plt.scatter(samples[n*10:n*10+10, 0], samples[n*10:n*10+10:, 1], c='r', s=5, label='Samples')
plt.scatter(mus[n, 0], mus[n, 1], c='b', s=20, label='current mu')
plt.scatter(mus[n + 1, 0], mus[n + 1, 1], c='w', s=20, label='next mu')
plt.title(f"sigmas:{sigmas[n,:]}")
plt.legend()

In [ ]:
import matplotlib.patches as patches

fig, ax = plt.subplots()
ax.set_ylim(0, 1)
ax.set_xlim(0, 1)
plt.contourf(X, Y, Loss_field, levels=100)
plt.colorbar()

patch = patches.Arrow(mus[0, 0], mus[0, 1], mu_grads[0, 0], mu_grads[0, 1], width=0.02, color='k')
scat1 = plt.scatter(samples[:, 0], samples[:, 1], c='r', s=5)
scat2 = plt.scatter(mus[:, 0], mus[:, 1], c='b', s=20)
def init():
    ax.add_patch(patch)
    return patch,

def update(num, mu, samples, scat1, scat2, ax):
    ax.patches.pop(0)
    patch = patches.Arrow(mus[num-2, 0], mus[num-2, 1], mu_grads[num-2, 0], mu_grads[num-2, 1], width=0.01, color='k')
    ax.add_patch(patch)
    scat1.set_offsets(samples[(num-2)*10: num*10, :])
    scat2.set_offsets(mu[num-2: num, :])
    return scat1,

ani = animation.FuncAnimation(fig, update, len(mus[:, 0]), fargs=[mus, samples, scat1, scat2, ax],
                              interval=10000, blit=False, init_func=init)


In [ ]:
N = -1
mlab.surf(X.T[:N, :N], Y.T[:N, :N], Loss_field.T[:N, :N])

In [ ]:
n=3
plt.contourf(X, Y, Loss_field, levels=100)
plt.scatter(samples[n*10:10+10*n ,0], samples[n*10:10+10*n,1], s=1.,c='r')
plt.arrow(mus[n,0], mus[n,1], mu_grads[n,0], mu_grads[n,1], length_includes_head=True, head_width=0.04, head_length=0.2)
plt.scatter(mus[n:n+2, 0], mus[n:n+2, 1])
plt.xlim((0, 1))
plt.ylim((0, 1))

In [ ]:
phis = np.linspace(0, 1, 1000)
losses = []
for phi1 in phis:
    exp.reset()
    exp.run_all([0.2, phi1], [0.25, 3000000])
    losses.append(exp.loss())
plt.plot(losses[:])

In [ ]:
plt.contourf(X, Y, Loss_field, levels=100)

In [ ]:
exp_time = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=None)
pol_time = PolicyTime(2, sigma_max = .2)
losses_time, best_program_time, mus_time, sigmas_time, n_par_time = reinforce_delta_tau(
        exp_time, 
        pol_time,
        num_episodes=5000, 
        batch_size=10,
        optim=torch.optim.SGD,
        lr=.1, 
        print_every=100,
        lr_decay=lambda a, b, c: step_decay(a, b, c, steps=5, decay_factor=0.65),
        baseline=0.7,
        max_norm=1.
    )

In [ ]:
exp_time.reset()
exp_time.run_all(pol_time.mu.detach().numpy()[:2], [0.25, 3000000])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
exp_time.reset()
exp_time.run_all(best_program_time[0], best_program_time[1])
exp_time.print_analytes(rc=(7, 5))
exp_time.loss()

In [ ]:
plt.plot(losses_time)

In [ ]:
plt.plot(mus_time[:, 0], label='Mu: 0')
plt.plot(mus_time[:, 1], label='Mu: 1')
plt.plot(mus_time[:, 2], label='Mu: 2')
plt.legend()
plt.show()

In [ ]:
plt.plot(sigmas_time[:, 0], label='Sigma: 0')
plt.plot(sigmas_time[:, 1], label='Sigma: 1')
plt.plot(sigmas_time[:, 2], label='Sigma: 2')
plt.legend()
plt.show()

In [ ]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
results = []
positions = []
taus = [.25, 3000000]
for n in range(300):

    pol = Policy(2, sigma_max = 0.2)

    losses, best_program, mus, sigmas, n_pars, samples, mu_grads, sigma_grads = reinforce(
            exp, 
            pol, 
            taus, 
            num_episodes=3000, 
            batch_size=10,
            optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65),
            lr=.1, 
            print_every=2999,
            lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=0.5),
            baseline=0.55,
            max_norm = 1.,
            #beta = 0.1
        )
    exp.reset()
    exp.run_all(pol.mu.detach().numpy(), taus)
    results.append(exp.loss())
    positions.append(pol.mu.detach())

In [ ]:
plt.hist(results, bins=100)
plt.title("Peter32, global min: 0.47")

In [ ]:

exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
pol = Policy(2, sigma_max = 0.2)
taus = [.25, 3000000]
base_dic = {}
for base in np.linspace(-1.5, 1.5, 20):
    r = 0
    for n in range(10):
        pol = Policy(2, sigma_max = 0.2)

        losses, best_program, mus, sigmas, n_pars, samples, mu_grads, sigma_grads = reinforce(
                exp, 
                pol, 
                taus, 
                num_episodes=3000, 
                batch_size=10,
                optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.9),
                lr=.1, 
                print_every=2999,
                lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=0.5),
                baseline=base,
                max_norm = 1.,
                #beta = 0.1
            )
        exp.reset()
        exp.run_all(pol.mu.detach().numpy(), taus)
        r += exp.loss()
    base_dic[base] = r/10

In [ ]:
base_dic

In [ ]:

exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
pol = Policy(2, sigma_max = 0.2)
taus = [.25, 3000000]
mom_dic = {}
for base in np.linspace(0., 1., 100):
    r = 0
    for n in range(10):
        pol = Policy(2, sigma_max = 0.2)

        losses, best_program, mus, sigmas, n_pars, samples, mu_grads, sigma_grads = reinforce(
                exp, 
                pol, 
                taus, 
                num_episodes=3000, 
                batch_size=10,
                optim=lambda a, b: torch.optim.SGD(a, b, momentum=base),
                lr=.1, 
                print_every=2999,
                lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=0.5),
                baseline=0.4,
                max_norm = 1.,
                #beta = 0.1
            )
        exp.reset()
        exp.run_all(pol.mu.detach().numpy(), taus)
        r += exp.loss()
    mom_dic[base] = r/10

In [23]:
exp = ExperimentAnalytes(k0 = alist.k0.values, S = alist.S.values, h=0.001, grad='iso', run_time=10.0)
results_n = []
for n in range(1,11):
    taus = np.ones(n) * 1/n
    result = []
    print(n)
    for i in range(10):
        pol = Policy(n, sigma_max = 0.2)
        
        reinforce(
                exp, 
                pol, 
                taus, 
                num_episodes=3000, 
                batch_size=10,
                optim=lambda a, b: torch.optim.SGD(a, b, momentum=0.65),
                lr=.1, 
                print_every=2999,
                lr_decay=lambda a, b, c: step_decay(a, b, c, steps=4, decay_factor=0.5),
                baseline=0.55,
                max_norm = 1.,
                #beta = 0.1
            )
        exp.reset()
        if n == 1:
            exp.run_all([pol.mu.detach().tolist()], taus)
        else:
            exp.run_all(pol.mu.detach().numpy(), taus)
        result.append(exp.loss())
    results_n.append(result)    
    

1
Loss: 0.2852055336550113, epoch: 2999/3000
Loss: 0.29259608188942104, epoch: 2999/3000
Loss: 0.2893002022962836, epoch: 2999/3000
Loss: 0.28906887897076083, epoch: 2999/3000
Loss: 0.2894069957821531, epoch: 2999/3000
Loss: 0.2843167524690778, epoch: 2999/3000
Loss: 0.2865874671017776, epoch: 2999/3000
Loss: 0.2840773413255688, epoch: 2999/3000
Loss: 0.29015899198121703, epoch: 2999/3000
Loss: 0.28713279057034974, epoch: 2999/3000
2
Loss: 0.2433500697652752, epoch: 2999/3000
Loss: 0.24903077534566945, epoch: 2999/3000
Loss: 0.2711005110970969, epoch: 2999/3000
Loss: 0.25013398978828527, epoch: 2999/3000
Loss: 0.26025238391956046, epoch: 2999/3000
Loss: 0.2565933474897162, epoch: 2999/3000
Loss: 0.23794384122699097, epoch: 2999/3000
Loss: 0.24669296356669432, epoch: 2999/3000
Loss: 0.23965576992751417, epoch: 2999/3000
Loss: 0.2584717639659456, epoch: 2999/3000
3
Loss: 0.20000671685279095, epoch: 2999/3000
Loss: 0.20389882824285363, epoch: 2999/3000
Loss: 0.19543189137277864, epoch: 29

In [24]:
plt.plot(range(1, 11), np.array(results_n).mean(axis=1))
plt.title("Expected loss vs Number of paramaters(GilarSample)")
plt.xlabel("N")
plt.ylabel("Expected loss")

Text(0, 0.5, 'Expected loss')